## Redispatch modelling using PyPSA

This tutorial demonstrates modelling and simulation of redispatch mechanism using PyPSA as a plug and play module in ASSUME-framework. The model will be created mainly taking grid constraints into consideration to identify grid bottlenecks with dispatches from EOM and resolve them using the redispatch algorithm.

### Concept of Redispatch:

The locational mismatch in demand and generation of electricity needs transmission of electricity from low demand regions to high demand regions. The transmission capacity limits the maximum amounts of electricity which can be transmitted at any point in time. If there is no enough capacity to transmit the required amount of electricity then there is a need of ramping down of generation at the locations of low demand and ramping up of generation at the locations of higher demand. This is typically called as Redispatch. Apart from spot markets there is redispatch mechanism to regulate this grid flows to avoid congestion issues. It is operated and controlled by the System operators (SO).

### Objective: 
The aim of redispatch is to reduce the overall cost of Redispatch(starting up, shuting down, ramping up, ramping down).

### Structure in Redispatch model:
- The redispatch has following structure:
    1. **Ramping up of reserved powerplants**:
    2. **Ramping up of market powerplants**
    2. **Ramping down of market powerplants**:
    3. **Ramping up/down of other flexibilites**:

---
### Objective of This Tutorial:
In this tutorial, we will:
1. Set up a **2-node** example of redispatch.
2. Connect hypothetical **generators**,**loads** and **transmission lines** to illustrate flow of energy.
3. Add **demand_side_units** to analyse their impact on overall redispatch.
4. Simulate and visualize the results.
---

## Setting Up the Simulation Environment

### A) Loads csv files from the given path and returns a dataframe

In [ ]:
# Simplified function to add read required CSV files
def read_grid(network_path: str | Path) -> dict[str, pd.DataFrame]:
    network_path = Path(network_path)
    buses = pd.read_csv(network_path / "buses.csv", index_col=0)
    lines = pd.read_csv(network_path / "lines.csv", index_col=0)
    generators = pd.read_csv(network_path / "powerplant_units.csv", index_col=0)
    loads = pd.read_csv(network_path / "demand_units.csv", index_col=0)

    return {
        "buses": buses,
        "lines": lines,
        "generators": generators,
        "loads": loads,
    }

### B) Simplified function to add generators to the grid network

In [12]:
# Simplified function to add generators to the grid network
def add_generators(
    network: pypsa.Network,
    generators: pd.DataFrame,
) -> None:
    """
    Add generators normally to the grid

    Args:
        network (pypsa.Network): the pypsa network to which the generators are
        generators (pandas.DataFrame): the generators dataframe
    """
    p_set = pd.DataFrame(
        np.zeros((len(network.snapshots), len(generators.index))),
        index=network.snapshots,
        columns=generators.index,
    )
    # add generators
    network.madd(
        "Generator",
        names=generators.index,
        bus=generators["node"],  # bus to which the generator is connected to
        p_nom=generators["max_power"],  # Nominal capacity of the powerplant/generator
        p_min_pu=p_set,
        p_max_pu=p_set + 1,
        marginal_cost=p_set,
        **generators,
    )

### C) Simplified function to add loads to the grid network

In [ ]:
# Simplified function to add loads to the grid network
def add_loads(
    network: pypsa.Network,
    loads: pd.DataFrame,
) -> None:
    """
    Add loads normally to the grid

    Args:
        network (pypsa.Network): the pypsa network to which the loads are
        loads (pandas.DataFrame): the loads dataframe
    """

    # add loads
    network.madd(
        "Load",
        names=loads.index,
        bus=loads["node"],  # bus to which the generator is connected to
        **loads,
    )

    if "p_set" not in loads.columns:
        network.loads_t["p_set"] = pd.DataFrame(
            np.zeros((len(network.snapshots), len(loads.index))),
            index=network.snapshots,
            columns=loads.index,
        )

### D) Simplified function to add loads to the redispatch network

In [ ]:
# Simplified function to add loads to the redispatch network
def add_redispatch_loads(
    network: pypsa.Network,
    loads: pd.DataFrame,
) -> None:
    """
    This adds loads to the redispatch PyPSA network with respective bus data to which they are connected
    """
    loads_c = loads.copy()
    if "sign" in loads_c.columns:
        del loads_c["sign"]

    # add loads with opposite sign (default for loads is -1). This is needed to properly model the redispatch
    network.madd(
        "Load",
        names=loads.index,
        bus=loads["node"],  # bus to which the generator is connected to
        sign=1,
        **loads_c,
    )

    if "p_set" not in loads.columns:
        network.loads_t["p_set"] = pd.DataFrame(
            np.zeros((len(network.snapshots), len(loads.index))),
            index=network.snapshots,
            columns=loads.index,
        )

In [ ]:
# Simplified function to add grid buses and lines to the redispatch network
def read_pypsa_grid(
    network: pypsa.Network,
    grid_dict: dict[str, pd.DataFrame],
):
    """
    Generates the pypsa grid from a grid dictionary.
    Does not add the generators, as they are added in different ways, depending on wether redispatch is used.

    Args:
        network (pypsa.Network): the pypsa network to which the components will be added
        grid_dict (dict[str, pd.DataFrame]): the dictionary containing dataframes for generators, loads, buses and links
    """

    def add_buses(network: pypsa.Network, buses: pd.DataFrame) -> None:
        network.import_components_from_dataframe(buses, "Bus")

    def add_lines(network: pypsa.Network, lines: pd.DataFrame) -> None:
        network.import_components_from_dataframe(lines, "Line")

    # setup the network
    add_buses(network, grid_dict["buses"])
    add_lines(network, grid_dict["lines"])
    return network

## Step 1: Setting up grid network with infrastructure

The grid infrastructure includes mainly three components:

- **Generators**: Used to produce hydrogen for steel production.
- **Loads**: Directly reduces iron ore using hydrogen.
- **Transmission grid**: Converts the reduced iron into steel.


Here the components are defined with their operational constraints (such as power, efficiency, ramp rates etc.)

In [ ]:
import pypsa
import numpy as np
import pandas as pd

## Step 2: Create a network and set Snapshots

In [ ]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24
solver='glpk'

### A) Define Nodes (here in PyPSA terminoloy Bus) 

In [ ]:
buses = pd.DataFrame(
    {
        "name": ["north_1", "north_2", "south"],
        "v_nom": [380.0, 380.0, 380.0],
        "zone_id": ["DE_1", "DE_1", "DE_2"],
        "x": [10.0, 9.5, 11.6],
        "y": [54.0, 53.5, 48.1],
    }
)

# Display the buses DataFrame
print("Buses DataFrame:")
display(buses)

In [ ]:
network.madd(
"Bus",
names
**buses
) 

### B) Define Powerplants (Generators in PyPSA)

In [ ]:
powerplant_units = pd.DataFrame(
    {
        "name": names,
        "technology": technology,
        "fuel_type": fuel_type,
        "emission_factor": emission_factor,
        "max_power": max_power,
        "min_power": min_power,
        "efficiency": efficiency,
        "additional_cost": additional_cost,
        "node": node,
        "unit_operator": unit_operator,
    }
)
display(powerplant_units.head())

In [ ]:
network.madd(
"Generator",

)

### C) Define the demand units (Loads in PyPSA)

In [ ]:
demand_units = pd.DataFrame(
    {
        "name": ["demand_north_1", "demand_north_2", "demand_south"],
        "technology": ["inflex_demand"] * 3,
        "bidding_zonal": ["naive_eom"] * 3,
        "max_power": [100000, 100000, 100000],
        "min_power": [0, 0, 0],
        "unit_operator": ["eom_de"] * 3,
        "node": ["north_1", "north_2", "south"],
    }
)

display(demand_units)

In [ ]:
network.madd(
"Load",

)

### D) Define Lines

In [ ]:
lines = pd.DataFrame(
    {
        "name": ["Line_N1_S", "Line_N2_S", "Line_N1_N2"],
        "bus0": ["north_1", "north_2", "north_1"],
        "bus1": ["south", "south", "north_2"],
        "s_nom": [5000.0, 5000.0, 5000.0],
        "x": [0.01, 0.01, 0.01],
        "r": [0.001, 0.001, 0.001],
    }
)
display(lines)

In [ ]:
network.madd(
"Line",

)

## Step 3: Solving to identify Network Congestion

In [ ]:
network.pf()
for i, (line_loading, s_nom_value) in enumerate(zip(round(network.lines_t.p0).values, network.lines.s_nom.values)):
    line_name = f"Line {i + 1}"
    if line_loading <= s_nom_value:
        congestion_status = False
        overloading = 0
        print("There is no congestion")
    else:
        congestion_status = True
        overloading = line_loading - s_nom_value
        print("The network is congested")

## Step 4: Redispatch Modelling